## 1.2 신경망의 추론
신경망을 복습해보자. 신경망에서 수행하는 작업은'학습'과 '추론' 두 단계로 나눌 수 있다.  
이번 절에서 '추론'에 대해 살펴본다

### 1.2.1 신경망 추론 전체 그림
신경망은 단순한 '함수'이다. 무엇을 입력하면 출력으로 변환한다.
2차원 데이터를 입력해 3차원 데이터를 출력하는 함수를 예로 들면 입력층에 뉴런 2개를, 출력층에 3개를 각 준비한다. 그리고 은닉층(중간층)에도 적당한 수의 뉴런을 배치한다.

그럼 이런 신경망 모습을 그릴 수 있다.  

       ㅇ ---> ㅇ ---> ㅇ  
       ㅇ ---> ㅇ ---> ㅇ  
              ㅇ ---> ㅇ    
              ㅇ
       input  hidden output
(그림상 이렇지만 각 노드들은 모두 서로 연결되어 있음)  
각 화살표에는 가중치가 존재하며, 그  가중치와 뉴런의 값을 각가 곱해서 그 합이 다음 뉴런의 입력으로 쓰인다. (정확하게는 그 값에 활성화 함수를 적용해서) 이때 각 층에서는 이전 뉴런값에 영향받지 않는 'bias' 편향 값도 더해진다.  이렇게 모든 뉴런이 연결되어 있는 신경망을 완전연결계층이라고 한다.

In [4]:
# N개의 데이터를 처리하는 완전연결계층 미니배치 구현
import numpy as np

W1 = np.random.randn(2,4)   # 가중치
b1 = np.random.randn(4)     # 편향
x = np.random.randn(10,2)   # 입력

h = np.matmul(x,W1) + b1    # 행렬곱

완전연결계층에 의한 변환은 '선형'변환이다. 여기에 '비선형'효과를 더하는 것이 활성화 함수이다. 여기서는 시그모이드 함수를 사용한다.

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
a = sigmoid(h)

In [7]:
# 최종정리
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# input
x = np.random.randn(10, 2)

# 1st layer
W1 = np.random.randn(2, 4)
b1 = np.random.randn(4)

# 2nd layer
W2 = np.random.randn(4, 3)
b2 = np.random.randn(3)

# forward
h = np.matmul(x, W1) + b1
a = sigmoid(h)
s = np.matmul(a, W2) + b2

### 1.2.2. 계층으로 클래스화 및 순전파 구현
신경망에서 하는 처리를 계층으로 구현해본다. 완전연결계층에 의한 변환을 Affine 계층으로, 시그모이드 함수에 의한 변환을 Sigmoid 계층으로 구현한다.
신경망을 구현할 때 다음의 구현 규칙을 따른다.
- 모든 계층은 forward()와 backward() 메서드를 갖는다.
- 모든 계층은 인스턴스 변수인 params와 grads를 갖는다.

이번절에서는 순전파를 구현한다.

In [8]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.params = []

    def forward(self, x):
        return 1 / (1 + np.exp(-x))

class Affine:
    def __init__(self, W, b):
        self.params = [W, b]

    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        return out

위의 클래스를 이용해 두개의 레이어를 가진 신경망을 구현

In [11]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size

        # 가중치와 편향 초기화
        W1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H, O)
        b2 = np.random.randn(O)

        # 계층 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]

        # 모든 가중치를 리스트에 모은다.
        self.params = []
        for layer in self.layers:
            self.params += layer.params

    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [17]:
x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)


In [18]:
model.params

[array([[ 1.00468848, -0.44554985,  0.94609825, -0.85659414],
        [ 0.27203115, -2.52312974, -0.61654743, -0.68147235]]),
 array([0.61096075, 1.07612968, 0.57846649, 1.40806188]),
 array([[ 0.30219731, -0.11782003, -1.34183137],
        [-0.51175314,  1.08172581, -0.68370835],
        [ 1.02615617, -1.11049703, -0.78652221],
        [-0.02441465,  0.85362359,  0.8113523 ]]),
 array([ 0.66136721, -1.02364077,  0.5404336 ])]

In [14]:
s.shape

(10, 3)